In [1]:
import sys
sys.path.append('../')
from scripts.graph import Station, RouteStopArr, RouteStopDep, WalkingStop, Timetable

In [2]:
from os.path import join
import pandas as pd

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
DATA = '../data/'

# init stations

In [5]:
%time
station_df = pd.read_csv(join(DATA, 'stations.csv'), index_col=0)

stations = {}
for i, row in station_df.iterrows():
    stations[row['STOP_NAME']]= Station(row['STOP_NAME'], row['STOP_NAME'], row['STOP_LAT'], row['STOP_LON'])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


# init routestops

In [61]:
arrivals = pd.read_csv(join(DATA, 'arrivals_ext.csv'), index_col=0).fillna(30)
terminus = pd.read_csv(join(DATA, 'terminus_ext.csv'), index_col=0)
departure = pd.read_csv(join(DATA, 'departures.csv'), index_col=0)

In [62]:
arrivals.head(2)

,STOP_NAME,stop_sequence,end_route_stop_id,route_stop_id,travel_time,target_end_route_stop_id,route_id,route_name,transport_type
0,"Zürich, Hardplatz",16,"26-72-j20-1$Zürich, Hardplatz$A","26-72-j20-1$Zürich, Hardplatz",30.0,"26-72-j20-1$Zürich, Albisriederplatz$D",26-72-j20-1,Bus 72,Bus
1,"Zürich, Zwinglihaus",11,"26-72-j20-1$Zürich, Zwinglihaus$A","26-72-j20-1$Zürich, Zwinglihaus",60.0,"26-72-j20-1$Zürich, Bertastrasse$D",26-72-j20-1,Bus 72,Bus


In [63]:
terminus.head(1)

,STOP_NAME,stop_sequence,end_route_stop_id,route_stop_id,route_id,route_name,transport_type
0,"Zürich, Triemlispital",1,"26-80-j20-1$Zürich, Triemlispital$A","26-80-j20-1$Zürich, Triemlispital",26-80-j20-1,Bus 80,Bus


In [86]:
len(terminus), len(departure), len(arrivals)

(968, 10901, 21543)

In [79]:

departure.head(2).values

array([['26-8-j20-1$Zürich, Bernoulli-Häuser', '8591074', '26-8-j20-1',
        'Zürich, Bernoulli-Häuser',
        '26-8-j20-1$Zürich, Bernoulli-Häuser$D', 0.0, 'Tram 8', 'Tram'],
       ['26-8-j20-1$Zürich, Langmauerstrasse', '8591244', '26-8-j20-1',
        'Zürich, Langmauerstrasse',
        '26-8-j20-1$Zürich, Langmauerstrasse$D', 0.0, 'Tram 8', 'Tram']],
      dtype=object)

In [81]:
routestops_arr, routestops_dep = {}, {}
for i, row in terminus.iterrows():
    routestops_arr[row['end_route_stop_id']] = RouteStopArr(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], row['stop_sequence'], 
                                                            row['route_name'], row['transport_type'], 0, None)
    
for i, row in arrivals.iterrows():
    routestops_arr[row['end_route_stop_id']] = RouteStopArr(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], row['stop_sequence'], 
                                                            row['route_name'], row['transport_type'], 
                                                            row['travel_time'], None)
    
for i, row in departure.iterrows():
    routestops_dep[row['end_route_stop_id']] = RouteStopDep(row['end_route_stop_id'],row['STOP_NAME'],
                                                            stations[row['STOP_NAME']], -1, 
                                                            row['route_name'], row['transport_type'], 
                                                            row['travel_time'], None)


for i, row in arrivals.iterrows():
    routestops_arr[ row['end_route_stop_id']].set_prev_stop(routestops_dep[row['target_end_route_stop_id']])
    
for i, row in departure.iterrows():
    current_id = row['end_route_stop_id']
    previous_id = current_id[:-2] + '$A'
    routestops_dep[current_id].set_prev_stop(routestops_arr[previous_id])

KeyError: '69-3-Y-j20-1$Reppischhof$A'

In [84]:
arrivals.end_route_stop_id.str.startswith("69-3-Y-j20-1$Reppisc").sum()

0

# init timetable

In [6]:
TARGET = 1.589388e+09

In [7]:
from datetime import datetime



In [8]:
# Dict[RouteStopDep, Tuple[List[int], List[Distrib]]]

table_df = pd.read_csv(join(DATA,  'timetableF.csv'))
table_df = table_df[table_df.end_route_stop_id.str.endswith('D')].copy()
table_df = table_df.drop_duplicates(['arrival_time_complete_unix', 'end_route_stop_id'])

In [11]:
hours = table_df.groupby("end_route_stop_id").apply(lambda row: list(row.arrival_time_complete_unix)).to_frame()

In [12]:
timetable = {}
for i, row in hours.iterrows():
    timetable[row.name] = {'time': row[0]}

In [14]:
timetable

{'1-1-C-j20-1$Zürich HB$D': {'time': [1589346120, 1589347140]},
 '1-1-D-j20-1$Dietikon$D': {'time': [1589332440]},
 '1-1-D-j20-1$Glanzenberg$D': {'time': [1589332200]},
 '1-1-D-j20-1$Killwangen-Spreitenbach$D': {'time': [1589332680]},
 '1-1-D-j20-1$Schlieren$D': {'time': [1589332080]},
 '1-1-D-j20-1$Zürich Altstetten$D': {'time': [1589331900]},
 '1-1-D-j20-1$Zürich HB$D': {'time': [1589331600]},
 '1-1-D-j20-1$Zürich Hardbrücke$D': {'time': [1589331720]},
 '1-1-j20-1$Würenlos, Bahnhof$D': {'time': [1589350440,
   1589380680,
   1589376720,
   1589406240,
   1589354220,
   1589368620,
   1589394720,
   1589388420,
   1589399580,
   1589348280,
   1589369880,
   1589390040,
   1589367720,
   1589355120,
   1589365380,
   1589393280,
   1589353680,
   1589382480,
   1589353320,
   1589398860,
   1589388060,
   1589383380,
   1589361240,
   1589406060,
   1589366460,
   1589391660,
   1589376180,
   1589382120,
   1589362680,
   1589348820,
   1589349180,
   1589383020,
   1589388240,
   15

# init walking

In [95]:
%%time
walking_df = pd.read_csv(join(DATA, 'walking_stops_pairs.csv'), index_col=0)
walking = {}

def get_id(station_name):
    return station_name + '_walk'

for station_name in walking_df.STOP_NAME.unique():
    if station_name in stations.keys():
        walk_id = get_id(station_name)
        walking[walk_id] = WalkingStop(walk_id, station_name, stations[station_name], [])
        
for i, row in walking_df.iterrows():
    if row['STOP_NAME'] in stations.keys() and row['STOP_NAME_2'] in stations.keys():
        walk_id_1 = get_id(row['STOP_NAME'])
        walk_id_2 = get_id(row['STOP_NAME_2'])
        walking[walk_id_1].add_neighbor((walking[walk_id_2], row['walk_time']))

CPU times: user 551 ms, sys: 79 µs, total: 551 ms
Wall time: 550 ms
